In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

#### Define Local LLM

In [2]:
from langchain_ollama import ChatOllama
llama_chat = ChatOllama(base_url="http//localhost:11434",
                        model="qwen2.5:latest",
                        temperature=0.5,
                        max_tokens=250)

#### Define Tools

In [ ]:
from langchain.tools import tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
@tool
def add_numbers(a: int, b: int):
    "Add 2 numbers and returns result"
    return a + b
@tool
def subtract_numbers(a: int, b: int):
    "subtracts 2 numbers and returns result"
    return a - b
@tool
def multiply_numbers(a: int, b: int):
    "Multiply 2 numbers and returns result"
    return a * b
wikipedia_tool = WikipediaQueryRun(api_wrapper= WikipediaAPIWrapper())

tools = [add_numbers, subtract_numbers, multiply_numbers, wikipedia_tool]

#### Test Tools using invoke

In [ ]:
print(add_numbers.invoke({"a":10, "b":20}))
print(subtract_numbers.invoke({"a":30, "b":10}))
print(multiply_numbers.invoke({"a":30, "b":7}))
print(wikipedia_tool.invoke("Explain Fibonacci series"))

In [16]:
## Store all tool name and details in a dictionary
list_of_tools = {tool.name: tool for tool in tools}
list_of_tools

{'add_numbers': StructuredTool(name='add_numbers', description='Add 2 numbers and returns result', args_schema=<class 'langchain_core.utils.pydantic.add_numbers'>, func=<function add_numbers at 0x10de3be20>),
 'subtract_numbers': StructuredTool(name='subtract_numbers', description='subtracts 2 numbers and returns result', args_schema=<class 'langchain_core.utils.pydantic.subtract_numbers'>, func=<function subtract_numbers at 0x10de3b1a0>),
 'multiply_numbers': StructuredTool(name='multiply_numbers', description='Multiply 2 numbers and returns result', args_schema=<class 'langchain_core.utils.pydantic.multiply_numbers'>, func=<function multiply_numbers at 0x10de3afc0>),
 'wikipedia': WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/jyotisahoo/VScode_Project/LangchainOllama/venv/lib/python3.12/site-packages/wikipedia/__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000))}

#### Add Tools to Local LLM and Invoke Tool Calls

In [25]:
local_chat_with_tools = llama_chat.bind_tools(tools)
response = local_chat_with_tools.invoke("When Avatar Movie released the ways of water")
print(response)
print(response.tool_calls)
response = local_chat_with_tools.invoke("What is the double of 2")
print(response)
print(response.tool_calls)

content='' additional_kwargs={} response_metadata={'model': 'qwen2.5:latest', 'created_at': '2025-12-15T16:48:54.681702Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3301209125, 'load_duration': 98675292, 'prompt_eval_count': 357, 'prompt_eval_duration': 1535066125, 'eval_count': 27, 'eval_duration': 1448445543, 'logprobs': None, 'model_name': 'qwen2.5:latest', 'model_provider': 'ollama'} id='lc_run--019b22ea-2eab-76e0-9c13-e6dd8b03ea2e-0' tool_calls=[{'name': 'wikipedia', 'args': {'query': 'Avatar: The Way of Water release date'}, 'id': '34322246-b1ed-4d97-a732-8cf5db400a16', 'type': 'tool_call'}] usage_metadata={'input_tokens': 357, 'output_tokens': 27, 'total_tokens': 384}
[{'name': 'wikipedia', 'args': {'query': 'Avatar: The Way of Water release date'}, 'id': '34322246-b1ed-4d97-a732-8cf5db400a16', 'type': 'tool_call'}]
content='To find the double of a number, we simply multiply it by 2. The double of 2 would be:\n\n\\[ 2 \\times 2 = 4 \\]\n\nSo, the double of 2 is 4. I

In [24]:
print((response.tool_calls[0].keys()))

dict_keys(['name', 'args', 'id', 'type'])


#### Tool Execution and Response Output

In [31]:
from langchain_core.messages import HumanMessage

query = "what is the sum of 2 and 22"
message = [HumanMessage(query)]
ai_message = local_chat_with_tools.invoke(message)
message.append(ai_message)
print(message)

[HumanMessage(content='what is the sum of 2 and 22', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2025-12-15T17:03:50.496255Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5079726625, 'load_duration': 103405542, 'prompt_eval_count': 360, 'prompt_eval_duration': 3338280583, 'eval_count': 27, 'eval_duration': 1425906168, 'logprobs': None, 'model_name': 'qwen2.5:latest', 'model_provider': 'ollama'}, id='lc_run--019b22f7-d2fe-7611-a5a1-7ae14cded31a-0', tool_calls=[{'name': 'add_numbers', 'args': {'a': 2, 'b': 22}, 'id': '26ce6873-3939-4338-9a58-4f8f377ffaa4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 360, 'output_tokens': 27, 'total_tokens': 387})]


In [32]:
for too_call in ai_message.tool_calls:
    tool_name = too_call['name']
    execute_tools = list_of_tools[tool_name]
    tool_invoke = execute_tools.invoke(too_call)
    message.append(tool_invoke)
message

[HumanMessage(content='what is the sum of 2 and 22', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2025-12-15T17:03:50.496255Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5079726625, 'load_duration': 103405542, 'prompt_eval_count': 360, 'prompt_eval_duration': 3338280583, 'eval_count': 27, 'eval_duration': 1425906168, 'logprobs': None, 'model_name': 'qwen2.5:latest', 'model_provider': 'ollama'}, id='lc_run--019b22f7-d2fe-7611-a5a1-7ae14cded31a-0', tool_calls=[{'name': 'add_numbers', 'args': {'a': 2, 'b': 22}, 'id': '26ce6873-3939-4338-9a58-4f8f377ffaa4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 360, 'output_tokens': 27, 'total_tokens': 387}),
 ToolMessage(content='24', name='add_numbers', tool_call_id='26ce6873-3939-4338-9a58-4f8f377ffaa4')]

In [33]:
final_output = llama_chat.invoke(message)
print(final_output.content)

The sum of 2 and 22 is 24.
